In [1]:
import geemap
import ee

In [2]:
ee.Initialize()

In [3]:
gadm_4 = ee.FeatureCollection("projects/ee-gopalpenny/assets/INDIA_SUBDISTRICT_11")

In [4]:
# gadm_4.filter(ee.Filter.inList('STATE_UT',['Bihar','Uttar Pradesh','Punjab','Haryana'])).size().getInfo()
subdists = gadm_4.filter(ee.Filter.inList('STATE_UT',['Bihar','Uttar Pradesh','Punjab','Haryana'])) #.size().getInfo()
# subdists = gadm_4.filter(ee.Filter.inList('STATE_UT',['Haryana'])) #.size().getInfo()
subdists.size().getInfo()

997

In [5]:
firms = ee.ImageCollection("FIRMS").select("T21")

firesVis = {
  'min': 325.0,
  'max': 400.0,
  'palette': ['red', 'orange', 'yellow'],
}

In [6]:
start_year = 2016
end_year = 2022
years = ee.List.sequence(start_year,end_year)
yr = years.get(0)

def getFires(yr):
    start = ee.Date.fromYMD(yr,11,1)
    end = ee.Date.fromYMD(yr,12,1)
    fires_november = firms.filterDate(start,end).reduce(ee.Reducer.max())
    fires_november = fires_november.set('year',yr)
    return fires_november

fires_november = ee.ImageCollection(years.map(getFires))
fires_november.first().getInfo()


{'type': 'Image',
 'bands': [{'id': 'T21_max',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'year': 2016, 'system:index': '0'}}

In [8]:

def reduce_fires(image):
    subdist_fires = image.reduceRegions(
      collection=subdists,
      reducer=ee.Reducer.count(),
      scale=1000,
    )
    subdist_fires = subdist_fires.map(lambda x: x.set('year',image.get('year')))
    return subdist_fires

fires = fires_november.map(reduce_fires).flatten()
# fires.first().getInfo()


In [11]:
task = ee.batch.Export.table.toDrive(
    collection=fires,
    description='fires',
    fileFormat='CSV',
    selectors=['DISTRICT','NAME','STATE_UT','count','SUB_ID','year'],
    fileNamePrefix='fires_count_n_india_' + str(start_year) + '_' + str(end_year),
    folder='GEE_Div',
    )
task.start()

In [33]:
m = geemap.Map()

m.setCenter(80, 20, 6)
m.addLayer(fires, firesVis, 'Firms')
m

Map(center=[20, 80], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…